In [1]:
# Plunkylib creates its files in the current working directory, so let's change to the parent directory if we're in the notebooks directory, since we want to use the default samples
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

# we also want to ensure we have a .env file that contains our API keys, so warn if we don't have that file present in this directory
if not os.path.exists(".env"):
    print("WARNING: No .env file found in this directory. Please create one with the API Key contents mentioned in the README.md file.")

In [2]:
from plunkylib import *
import dataclasses


In [3]:
# with datafiles, you can look up an existing datafile object by name

# lets mimic the readme but using the objects instead of the CLI
example_params_to_copy = CompletionParams.objects.get("ExampleGPT3")
print(example_params_to_copy)

# make a copy and replace the name
params_myfirstengine = dataclasses.replace(example_params_to_copy, name="MyFirstEngine")
print(params_myfirstengine)

CompletionParams(name='ExampleGPT3', engine='text-davinci-002', n=1, best_of=1, max_tokens=300, stop=['\n\n', '----\n'], temperature=0.3, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0)
CompletionParams(name='MyFirstEngine', engine='text-davinci-002', n=1, best_of=1, max_tokens=300, stop=['\n\n', '----\n'], temperature=0.3, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0)


In [4]:
# Bonus: If you wanted, you can change the parameters and the file will change accordingly
params_myfirstengine.max_tokens = 100
print(params_myfirstengine)

CompletionParams(name='MyFirstEngine', engine='text-davinci-002', n=1, best_of=1, max_tokens=100, stop=['\n\n', '----\n'], temperature=0.3, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0)


In [5]:
# same thing with the prompt
example_prompt_to_copy = Prompt.objects.get("ExampleSimple")
print(example_prompt_to_copy)

# make a copy and replace the name
prompt_myfirstprompt = dataclasses.replace(example_prompt_to_copy, name="MyFirstPrompt")
prompt_myfirstprompt

Prompt(name='ExampleSimple', text='Describe the color of the sky in flowery language:\n----\nDESCRIPTION:\n')


Prompt(name='MyFirstPrompt', text='Describe the color of the sky in flowery language:\n----\nDESCRIPTION:\n')

In [6]:
# now a new Petition using the names of the prompt and params
petition_example = Petition("MyFirstPetition", prompt_myfirstprompt.name, params_myfirstengine.name)
petition_example

Petition(name='MyFirstPetition', prompt_name='MyFirstPrompt', params_name='MyFirstEngine', promptvars_name=None)

In [7]:
# petitions have some lazy references that we tend to load right after if we want to complete with them
petition_example.load_all()

example_completion, adj_prompt_text = await petition_completion2(petition_example, {})

2022-06-20 10:56:48.895 | DEBUG    | chronological:_completion:36 - CONFIG:
    Prompt: 'Describe the color of the sky in flowery language:\n----\nDESCRIPTION:\n'
    Temperature: 0.3
    Engine: text-davinci-002
    Max Tokens: 100
    Top-P: 1.0
    Stop: ['\n\n', '----\n']
    Presence Penalty 0.0
    Frequency Penalty: 0.0
    Echo: False
    N: 1
    Stream: False
    Log-Probs: None
    Best Of: 1
    Logit Bias: {}
2022-06-20 10:56:50.236 | DEBUG    | chronological:_completion:66 - GPT-3 Completion Result: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nThe sky is a beautiful blue color. It looks like a big blue ocean that you could dive into."
    }
  ],
  "created": 1655740609,
  "id": "cmpl-5LE3djoubDPaKAw3GcBhGN6nyyKcl",
  "model": "text-davinci-002",
  "object": "text_completion"
}


In [8]:
# Completion object contains our results
print(example_completion.text)

The sky is a beautiful blue color. It looks like a big blue ocean that you could dive into.


In [9]:
# We can change the prompt, if we want
prompt_myfirstprompt.text = "Describe, in great detail, the smell of the beach after an oil spill.\n\nDESCRIPTION:\n"

In [10]:
#And you don't need to rebuild a new petition, the original one works great
#You keep reusing it, and you can edit the prompt (or engine parameters) from code or from the text/yaml files
example_completion2, adj_prompt_text = await petition_completion2(petition_example, {})

# Completion object contains our results
print(example_completion2.text)

2022-06-20 10:57:07.808 | DEBUG    | chronological:_completion:36 - CONFIG:
    Prompt: 'Describe, in great detail, the smell of the beach after an oil spill.\n\nDESCRIPTION:\n'
    Temperature: 0.3
    Engine: text-davinci-002
    Max Tokens: 100
    Top-P: 1.0
    Stop: ['\n\n', '----\n']
    Presence Penalty 0.0
    Frequency Penalty: 0.0
    Echo: False
    N: 1
    Stream: False
    Log-Probs: None
    Best Of: 1
    Logit Bias: {}
2022-06-20 10:57:10.886 | DEBUG    | chronological:_completion:66 - GPT-3 Completion Result: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\nThe beach smells like a mixture of oil and salt water. There is a strong smell of petroleum and a hint of seaweed. The oil has a thick, greasy texture and clings to the sand."
    }
  ],
  "created": 1655740628,
  "id": "cmpl-5LE3w5Hhc7duHrQNYDP7EeyxncYvx",
  "model": "text-davinci-002",
  "object": "text_completion"
}


The beach smells like a mixture of oil and salt water. There is a strong smell of petroleum and a hint of seaweed. The oil has a thick, greasy texture and clings to the sand.


In [11]:
# We could run the same thing with Jurassic, we just need new CompletionParams and a new petition that uses our prompt
sample_jparams = CompletionParams.objects.get("ExampleJurassicJumbo")
params_mysecondengine = dataclasses.replace(sample_jparams, name="MySecondEngine")
params_mysecondengine.max_tokens = 100

# We don't need a new prompt, we can use both
petition_example2 = Petition("MySecondPetition", prompt_myfirstprompt.name, params_mysecondengine.name)
petition_example2

Petition(name='MySecondPetition', prompt_name='MyFirstPrompt', params_name='MySecondEngine', promptvars_name=None)

In [12]:
params_mysecondengine.max_tokens = 100
example_completion3, adj_prompt_text = await petition_completion2(petition_example2, {})

# Completion object contains our results
print(example_completion3.text)

The beach smells like oil. It smells like oil, and dead fish, and dead birds, and dead crabs, and dead dolphins, and dead whales, and dead people. It smells like oil, and dead fish, and dead birds, and dead crabs, and dead dolphins, and dead whales, and dead people. It smells like oil, and dead fish, and dead birds, and dead crabs, and dead dolphins, and dead whales, and dead people. It smells like oil, and dead fish, and dead
